## Base Model

Model: line-corporation/japanese-large-lm-3.6b
https://huggingface.co/line-corporation/japanese-large-lm-3.6b


In [ ]:
!pip install transformers accelerate sentencepiece peft bitsandbytes einops --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.0 MB/s eta 0:00:00


In [ ]:
# Wrap printed text
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained("line-corporation/japanese-large-lm-3.6b", use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    "line-corporation/japanese-large-lm-3.6b",
    trust_remote_code=True,
    quantization_config=quantization_config,
    device_map="auto",
).eval()

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Some weights of GPTNeoXForCausalLM were not initialized from the model checkpoint at line-corporation/japanese-large-lm-3.6b and are newly initialized: ['embed_out.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer.vocab_size

51200

In [ ]:
prompt = """
### 指示:
坂本龍馬について詳細に教えてください。

### 応答:

""".strip()

inputs = tokenizer(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

tokens = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.1,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.1,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 指示:坂本龍馬について詳細に教えてください。### 応答:Sakamoto Ryoma is a very good leader in Japan. Please tell me more about them. Thank you for your interest. #### 指示:坂本龍馬について詳細に教えてください。


In [ ]:
text = """
例題: りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう？
回答:
""".strip()
inputs = tokenizer(text, return_tensors='pt', )

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)


例題: りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう?回答: ウェブ<0x0A>本文:  10月3日(木)、4日(金)の2日間、東京ビッグサイトにて「第6回 国際物流総合展」が開催され、弊社も出展いたしました。 ご来場いただきました皆様、誠にありがとうございました。 展示会の様子は、後日改めてご紹介させていただきます。 さて、今回は、先日開催した「第7回 物流改善事例発表会」についてご報告いたします。 「物流改善事例発表会」とは、物流改善に取り組まれている企業様や団体様の発表


In [ ]:
text = """
例題: バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう？
回答:
""".strip()

inputs = tokenizer(text, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)


例題: バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう?回答: ウェブ<0x0A>本文:  2. ボウルに卵を割り入れ、泡立て器で溶きほぐします。 3. 鍋に牛乳を入れて中火にかけ、沸騰したら火から下ろして2を少しずつ加えながら混ぜます。 4. 再び鍋に入れて弱火にかけ、木べらで絶えずかき混ぜながらとろみがつくまで加熱します。 5. 火から下ろし、バニラエッセンスを加えて混ぜ合わせます。 6. 別のボウルに生クリームを入れ、氷水にあてて八分


In [ ]:
text = """
例題: 引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。
回答:
""".strip()

inputs = tokenizer(text, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)


例題: 引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。回答: Q&A<0x0A>Q : ヤフオクで、落札した商品がコンビニ受け取りを選べるのですが、コンビニ受け取りの場合、出品者に何か連絡する必要はありますか? A : コンビニ受取はヤフーが指定したコンビニのみ対応です。<0x0A>コンビニ受取を選択すると、落札者には「発送連絡」の次の画面で、「店舗受取り先を登録してください」というメッセージが表示されますので、その画面にある「店舗受取り先を登録する」ボタンを押します。<0x0A>次に、その画面の「受取方法を設定する」ボタンを押して、表示された画面で


In [ ]:
text = """
English: There were 3 apples and 2 oranges. How many fruits were there in total?
日本語:
""".strip()

inputs = tokenizer(text, return_tensors='pt')

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


English: There were 3 apples and 2 oranges. How many fruits were there in total?<unk>日本語:</s> ウェブ<0x0A>本文:  10月1日(火)から、10月14日(月・祝)まで、 10月15日(火)からは、通常通り営業いたします。 よろしくお願いいたします。 Tweet 2019年08月27日 こんにちは。 いつもありがとうございます。 お盆のお休みは、8月11日から16日まででした。 17日より通常営業しております。 よろしくお願いいたします。 Tweet 2019年


## Instruction Model

Model: line-corporation/japanese-large-lm-3.6b-instruction-sft
https://huggingface.co/line-corporation/japanese-large-lm-3.6b-instruction-sft

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained("line-corporation/japanese-large-lm-3.6b-instruction-sft", use_fast=False, padding_side='left')
model = AutoModelForCausalLM.from_pretrained(
    "line-corporation/japanese-large-lm-3.6b-instruction-sft",
    trust_remote_code=True,
    quantization_config=quantization_config,
)

In [ ]:
tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.pad_token_id

(1, 2, 2)

In [ ]:
tokenizer.decode(1), tokenizer.decode(2)

('<s>', '</s>')

In [ ]:
text = """
りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう？
""".strip()
inputs = tokenizer(f"ユーザー: {text}\nシステム: 一歩ずつ考えましょう。", return_tensors='pt', truncation=True, add_special_tokens=False) # add_special_tokens=False が重要。これがないと生成がされない。

model.eval()
with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_length=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        top_k=0,
        repetition_penalty=1.1,
        num_beams=1,
        pad_token_id=tokenizer.pad_token_id,
        num_return_sequences=1,
    )

output = tokenizer.decode(output_ids[0], skip_special_tokens=False)
print(output)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ユーザー: りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう?
システム: 一歩ずつ考えましょう。  あなたは5つりんごを持っています、それからあなたは2つのりんごを失いました。このことから、4-2で残りは2つです。</s>


In [ ]:
from transformers import pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

input_text = """りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう？"""
text = generator(
    f"ユーザー: {input_text}\nシステム: ",
    max_length = 256,
    do_sample = True,
    temperature = 0.7,
    top_p = 0.9,
    top_k = 0,
    repetition_penalty = 1.1,
    num_beams = 1,
    pad_token_id = tokenizer.pad_token_id,
    num_return_sequences = 1,
)
print(text)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'generated_text': 'ユーザー: りんごが5つあります。そこから2つのりんごを取り除きました。残りのりんごの数は何個でしょう？\nシステム:  りんごは5つある。\nそのうちの2つは、リンゴから取り除かれたものです。\n残りは4つです。'}]


In [ ]:
text = """バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう？""".strip()

inputs = tokenizer(f"ユーザー: {text}\nシステム: ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)


ユーザー: バットとボールの両方を買うと1100円です。バットはボールよりも1000円高いです。ボールはいくらでしょう?
システム: 1100-1000=100なので、ボールの代金は1000円になります。


In [ ]:
text = """
引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。
""".strip()

inputs = tokenizer(f"ユーザー: {text}\nシステム: ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True)
print(output)


ユーザー: 引数kを取り、返り値としてフィボナッチ数列におけるk個目の値を返すPython関数を書いてください。
システム: def fib_divisor(n):
    n> kの場合:
        return ord(ord(n%2) + 1, n)#falseを返します


In [ ]:
text = """
下記の英語を日本語に翻訳してください。
English: There were 3 apples and 2 oranges. How many fruits were there in total?
""".strip()

inputs = tokenizer(f"ユーザー: {text}\nシステム: ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


ユーザー: 下記の英語を日本語に翻訳してください。
English: There were 3 apples and 2 oranges. How many fruits were there in total?
システム: 3つのリンゴと2つのオレンジが1つありました。合計でいくつの果物がありましたか?</s>


In [ ]:
text = """
下記の文章を100字以内に要約してください。

```
【速報】ロシア月探査機「ルナ25」が月に衝突 「消滅した」ロスコスモス発表
月に向かっていたロシアの無人探査機「ルナ25」が月に衝突したことが分かった。ロシアの国営宇宙企業ロスコスモスは先ほど、「月に衝突し、消滅した」と明らかにした。月面着陸前の軌道に移行中、制御不能となったという。
探査機は21日に月の南極付近に着陸予定だった。
```
""".strip()

inputs = tokenizer(f"ユーザー: {text}\nシステム: ", return_tensors='pt', add_special_tokens=False)

with torch.no_grad():
    output_ids = model.generate(
        inputs['input_ids'].to(model.device),
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)


ユーザー: 下記の文章を100字以内に要約してください。

```
【速報】ロシア月探査機「ルナ25」が月に衝突 「消滅した」ロスコスモス発表
月に向かっていたロシアの無人探査機「ルナ25」が月に衝突したことが分かった。ロシアの国営宇宙企業ロスコスモスは先ほど、「月に衝突し、消滅した」と明らかにした。月面着陸前の軌道に移行中、制御不能となったという。
探査機は21日に月の南極付近に着陸予定だった。
```
システム: 月への有人ミッションの成功に向けて、月面に着陸する予定だったロシアの無人探査機「ルナ25」が、月に向かっていた途中で制御不能となり、月面に衝突しました。</s>
